## 5400 Group 1 - NYPD Calls for Service Data

### Jasmine(Keyi) Jiang, Heyou Pan, Jiajun Ma, Haonan Yao, Tianren Xie
### Technology: Python Flask, Postgre SQL, MongoDB

## 1. Data Loading & Preprocessing

In [1]:
import pandas as pd

In [2]:
%%time

#partially preprocessed in R by adding efficiency columns

df = pd.read_csv('5400NYPD.csv')
df.head()

CPU times: user 23.1 s, sys: 4.6 s, total: 27.7 s
Wall time: 29.8 s


,Unnamed: 0,CAD_EVNT_ID,CREATE_DATE,INCIDENT_DATE,INCIDENT_TIME,NYPD_PCT_CD,BORO_NM,PATRL_BORO_NM,GEO_CD_X,GEO_CD_Y,...,DISP_TS,ARRIVD_TS,CLOSNG_TS,Latitude,Longitude,INCIDENT_TS,OVERALL_EFF,RESPOND_EFF,DISPATCH_EFF,SOLVE_EFF
0,1,82283653,01/01/2022,12/31/2021,23:02:02,42.0,BRONX,BRONX,1014646,243583,...,01/01/2022 12:19:23 AM,NaN,01/01/2022 01:21:48 AM,40.835210,-73.890157,12/31/2021 23:02:02,8386.0,4324,317,3745.0
1,2,82284236,01/01/2022,12/31/2021,23:42:04,18.0,MANHATTAN,MAN SOUTH,989529,218317,...,01/01/2022 12:03:41 AM,NaN,01/01/2022 12:58:08 AM,40.765913,-73.980944,12/31/2021 23:42:04,4564.0,1242,55,3267.0
2,3,82284489,01/01/2022,01/01/2022,00:00:16,121.0,STATEN ISLAND,STATEN ISLAND,939031,165829,...,01/01/2022 12:00:21 AM,NaN,01/01/2022 01:27:28 AM,40.621732,-74.162890,01/01/2022 00:00:16,5232.0,0,5,5227.0
3,4,82284490,01/01/2022,01/01/2022,00:00:23,70.0,BROOKLYN,BKLYN SOUTH,992635,174415,...,01/01/2022 02:06:26 AM,NaN,01/01/2022 02:43:48 AM,40.645410,-73.969786,01/01/2022 00:00:23,9805.0,0,7563,2242.0
4,5,82284493,01/01/2022,01/01/2022,00:00:33,44.0,BRONX,BRONX,1008846,245939,...,01/01/2022 12:46:20 AM,NaN,01/01/2022 04:57:39 AM,40.841695,-73.911108,01/01/2022 00:00:33,17826.0,0,2747,15079.0


In [3]:
len(df)

7401019

In [4]:
sum(df.NYPD_PCT_CD.isna())

4

### 1.1 Manipulate original dataset to extract information for SQL database use.

In [5]:
#Isolate Borough and NYPE Precinct information from original dataset

df_boro = df.loc[:,['BORO_NM','PATRL_BORO_NM','NYPD_PCT_CD']].drop_duplicates(keep = 'first').dropna()

In [6]:
df_boro['NYPD_PCT_CD'] = df_boro['NYPD_PCT_CD'].astype(int)
df_boro = df_boro[df_boro['NYPD_PCT_CD'] != 0]

In [7]:
df_boro

,BORO_NM,PATRL_BORO_NM,NYPD_PCT_CD
0,BRONX,BRONX,42
1,MANHATTAN,MAN SOUTH,18
2,STATEN ISLAND,STATEN ISLAND,121
3,BROOKLYN,BKLYN SOUTH,70
4,BRONX,BRONX,44
...,...,...,...
278,MANHATTAN,MAN NORTH,20
333,QUEENS,QUEENS NORTH,109
355,MANHATTAN,MAN NORTH,33
388,BROOKLYN,BKLYN SOUTH,76


### 1.2 Load NYPD Information dataset

In [8]:
df_nypd = pd.read_csv('NYPD_INFO.csv')
df_nypd.head()

,NYPD_FULL_NAME,NYPD_PCT_CD,PHONE_NUMBER,ADDRESS
0,1st Precinct,1,212-334-0611,16 Ericsson Place
1,5th Precinct,5,212-334-0711,19 Elizabeth Street
2,6th Precinct,6,212-741-4811,233 West 10 Street
3,7th Precinct,7,212-477-7311,19 1/2 Pitt Street
4,9th Precinct,9,212-477-7811,321 East 5 Street


In [9]:
df_nypd['NYPD_PCT_CD'] = df_nypd['NYPD_PCT_CD'].astype(int)

### 1.3 Merge Borough NYPD data with NYPD information dataset and produce a new data table for NYPD precinct:
The table contains Borough, Patrol Borough, NYPD precinct ID, NYPD full name, contact phone number, and address.
This will be store in RDBMS.

In [10]:
df_sql = df_boro.merge(df_nypd, how = 'left', on = 'NYPD_PCT_CD')
df_sql

,BORO_NM,PATRL_BORO_NM,NYPD_PCT_CD,NYPD_FULL_NAME,PHONE_NUMBER,ADDRESS
0,BRONX,BRONX,42,42nd Precinct,718-402-3887,830 Washington Avenue
1,MANHATTAN,MAN SOUTH,18,Midtown North Precinct,212-767-8400,306 West 54th Street
2,STATEN ISLAND,STATEN ISLAND,121,121st Precinct,718-697-8700,970 Richmond Avenue
3,BROOKLYN,BKLYN SOUTH,70,70th Precinct,718-851-5511,154 Lawrence Avenue
4,BRONX,BRONX,44,44th Precinct,718-590-5511,2 East 169th Street
...,...,...,...,...,...,...
72,MANHATTAN,MAN NORTH,20,20th Precinct,212-580-6411,120 West 82nd Street
73,QUEENS,QUEENS NORTH,109,109th Precinct,718-321-2250,37-05 Union Street
74,MANHATTAN,MAN NORTH,33,33rd Precinct,212-927-3200,2207 Amsterdam Avenue
75,BROOKLYN,BKLYN SOUTH,76,76th Precinct,718-834-3211,191 Union Street


## 2. Connect to PostgreSQL database and construct data table

In [11]:
import psycopg, os

print('Connecting to the PostgreSQL database...')
conn = psycopg.connect(
    host="localhost",
    port='5432',
    dbname="postgres",
    user="postgres",
    password="123")

Connecting to the PostgreSQL database...


In [12]:
cur = conn.cursor()

In [31]:
#conn.rollback()

In [13]:
cur.execute("DROP TABLE NYPD_PCT")

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost database=postgres) at 0x7f864810c3b0>

### 2.1 Create and insert into Table NYPD_PCT

In [14]:
# Create Table NYPD_PCT

createCmd = """ CREATE TABLE NYPD_PCT (
                BORO_NM VARCHAR(25),
                PATRL_BORO_NM VARCHAR(255),
                NYPD_PCT_CD INT NOT NULL,
                NYPD_FULL_NAME VARCHAR(255),
                PHONE_NUMBER VARCHAR(15),
                ADDRESS VARCHAR(255),
                Primary Key(NYPD_PCT_CD)
                )
            """
    
cur.execute(createCmd)
conn.commit()


In [19]:
%%time

# Insert all rows in the previous dataframe df_sql into Table NYPD_PCT

for row in df_sql.itertuples():
    cur.execute('''
                INSERT INTO NYPD_PCT 
                (BORO_NM, PATRL_BORO_NM, 
                NYPD_PCT_CD, NYPD_FULL_NAME, 
                PHONE_NUMBER, ADDRESS)
                VALUES (%s,%s,%s,%s,%s,%s)
                ''',
                (row.BORO_NM, 
                row.PATRL_BORO_NM,
                row.NYPD_PCT_CD,
                row.NYPD_FULL_NAME,
                row.PHONE_NUMBER,
                row.ADDRESS)
                )
    
conn.commit()

CPU times: user 14.4 ms, sys: 7.79 ms, total: 22.1 ms
Wall time: 146 ms


In [20]:
#select and print all from the table

queryCmd = 'SELECT * from NYPD_PCT'    
cur.execute(queryCmd)    
print("The number of rows: ", cur.rowcount)

row = cur.fetchone()
while row is not None:
    print(row)
    row = cur.fetchone()    

The number of rows:  77
('BRONX', 'BRONX', 42, '42nd Precinct', '718-402-3887', '830 Washington Avenue')
('MANHATTAN', 'MAN SOUTH', 18, 'Midtown North Precinct', '212-767-8400', '306 West 54th Street')
('STATEN ISLAND', 'STATEN ISLAND', 121, '121st Precinct', '718-697-8700', '970 Richmond Avenue')
('BROOKLYN', 'BKLYN SOUTH', 70, '70th Precinct', '718-851-5511', '154 Lawrence Avenue')
('BRONX', 'BRONX', 44, '44th Precinct', '718-590-5511', '2 East 169th Street')
('MANHATTAN', 'MAN SOUTH', 10, '10th Precinct', '212-741-8211', '230 West 20th Street')
('BRONX', 'BRONX', 49, '49th Precinct', '718-918-2000', '2121 Eastchester Road')
('MANHATTAN', 'MAN SOUTH', 14, 'Midtown South Precinct', '212-239-9811', '357 West 35th Street')
('MANHATTAN', 'MAN NORTH', 19, '19th Precinct', '212-452-0600', '153 East 67th Street')
('BROOKLYN', 'BKLYN NORTH', 77, '77th Precinct', '718-735-0611', '127 Utica Avenue')
('BRONX', 'BRONX', 40, '40th Precinct', '718-402-2270', '257 Alexander Avenue')
('BROOKLYN', 'B

### 2.2 Execute SQL Query

The SQL query takes in two variables filtering Boro_nm and Patrl_boro_nm, it will return all the NYPD precincts within the searching region. 
For example, if we search for the North side of Queens, it will return 8 NYPD precincts in the area.

In [22]:
queryCmd = "Select * from NYPD_PCT where BORO_NM LIKE '%BRONX%'"    
cur.execute(queryCmd) 
print("The number of rows: ", cur.rowcount)

row = cur.fetchone()
while row is not None:
    print(row)
    row = cur.fetchone() 

The number of rows:  12
('BRONX', 'BRONX', 42, '42nd Precinct', '718-402-3887', '830 Washington Avenue')
('BRONX', 'BRONX', 44, '44th Precinct', '718-590-5511', '2 East 169th Street')
('BRONX', 'BRONX', 49, '49th Precinct', '718-918-2000', '2121 Eastchester Road')
('BRONX', 'BRONX', 40, '40th Precinct', '718-402-2270', '257 Alexander Avenue')
('BRONX', 'BRONX', 43, '43rd Precinct', '718-542-0888', '900 Fteley Avenue')
('BRONX', 'BRONX', 41, '41st Precinct', '718-542-4771', '1035 Longwood Avenue')
('BRONX', 'BRONX', 47, '47th Precinct', '718-920-1211', '4111 Laconia Avenue')
('BRONX', 'BRONX', 52, '52nd Precinct', '718-220-5811', '3016 Webster Avenue')
('BRONX', 'BRONX', 48, '48th Precinct', '718-299-3900', '450 Cross Bronx Expressway')
('BRONX', 'BRONX', 50, '50th Precinct', '718-543-5700', '3450 Kingsbridge Avenue')
('BRONX', 'BRONX', 46, '46th Precinct', '718-220-5211', '2120 Ryer Avenue')
('BRONX', 'BRONX', 45, '45th Precinct', '718-822-5411', '2877 Barkley Avenue')


In [23]:
queryCmd = "Select * from NYPD_PCT where BORO_NM LIKE '%QUEENS%' and PATRL_BORO_NM LIKE '%NORTH%'"    
cur.execute(queryCmd) 
print("The number of rows: ", cur.rowcount)

row = cur.fetchone()
while row is not None:
    print(row)
    row = cur.fetchone() 

The number of rows:  8
('QUEENS', 'QUEENS NORTH', 104, '104th Precinct', '718-386-3004', '64-2 Catalpa Avenue')
('QUEENS', 'QUEENS NORTH', 114, '114th Precinct', '718-626-9311', '34-16 Astoria Boulevard')
('QUEENS', 'QUEENS NORTH', 111, '111th Precinct', '718-279-5200', '45-06 215th Street')
('QUEENS', 'QUEENS NORTH', 112, '112th Precinct', '718-520-9311', '68-40 Austin Street')
('QUEENS', 'QUEENS NORTH', 110, '110th Precinct', '718-476-9311', '94-41 43rd Avenue')
('QUEENS', 'QUEENS NORTH', 115, '115th Precinct', '718-533-2002', '92-15 Northern Boulevard')
('QUEENS', 'QUEENS NORTH', 108, '108th Precinct', '718-784-5411', '5-47 50th Avenue')
('QUEENS', 'QUEENS NORTH', 109, '109th Precinct', '718-321-2250', '37-05 Union Street')


In [24]:
%%time

# Assign variables 1 and 2 and implement them as filters in the SQL query

var1 = 'QUEENS' 
var2 = 'NORTH' 

queryCmd_3 = "Select * from NYPD_PCT where BORO_NM LIKE '%%%s%%' and PATRL_BORO_NM LIKE '%%%s%%'"  % ( var1, var2 )  
cur.execute(queryCmd_3) 
print("The number of rows: ", cur.rowcount)

row = cur.fetchone()
while row is not None:
    print(row)
    row = cur.fetchone() 

The number of rows:  8
('QUEENS', 'QUEENS NORTH', 104, '104th Precinct', '718-386-3004', '64-2 Catalpa Avenue')
('QUEENS', 'QUEENS NORTH', 114, '114th Precinct', '718-626-9311', '34-16 Astoria Boulevard')
('QUEENS', 'QUEENS NORTH', 111, '111th Precinct', '718-279-5200', '45-06 215th Street')
('QUEENS', 'QUEENS NORTH', 112, '112th Precinct', '718-520-9311', '68-40 Austin Street')
('QUEENS', 'QUEENS NORTH', 110, '110th Precinct', '718-476-9311', '94-41 43rd Avenue')
('QUEENS', 'QUEENS NORTH', 115, '115th Precinct', '718-533-2002', '92-15 Northern Boulevard')
('QUEENS', 'QUEENS NORTH', 108, '108th Precinct', '718-784-5411', '5-47 50th Avenue')
('QUEENS', 'QUEENS NORTH', 109, '109th Precinct', '718-321-2250', '37-05 Union Street')
CPU times: user 1.88 ms, sys: 1.6 ms, total: 3.48 ms
Wall time: 4.38 ms


In [95]:
# cur.close()

## 3. Connect to MongoDB database and construct collection for NYPD Calls

### 3.1 Connect to MongoDB and create apan 5400 Database

In [25]:
from pymongo import MongoClient
client = MongoClient('localhost',27017) ## or MongoClient("localhost:27017")
db = client.apan5400

### 3.2 Preprocess NYPD call event info as documents

In [6]:
%%time
import csv
myFile = open('5400NYPD.csv', 'r')
reader = csv.DictReader(myFile)
newsfeeds = list(reader)

CPU times: user 46.4 s, sys: 4.58 s, total: 51 s
Wall time: 55.1 s


In [7]:
newsfeeds[0]

{'': '1',
 'CAD_EVNT_ID': '82283653',
 'CREATE_DATE': '01/01/2022',
 'INCIDENT_DATE': '12/31/2021',
 'INCIDENT_TIME': '23:02:02',
 'NYPD_PCT_CD': '42',
 'BORO_NM': 'BRONX',
 'PATRL_BORO_NM': 'BRONX',
 'GEO_CD_X': '1014646',
 'GEO_CD_Y': '243583',
 'RADIO_CODE': '54E1',
 'TYP_DESC': 'AMBULANCE CASE: EDP/INSIDE',
 'CIP_JOBS': 'Non CIP',
 'ADD_TS': '01/01/2022 12:14:06 AM',
 'DISP_TS': '01/01/2022 12:19:23 AM',
 'ARRIVD_TS': 'NA',
 'CLOSNG_TS': '01/01/2022 01:21:48 AM',
 'Latitude': '40.83521',
 'Longitude': '-73.890157',
 'INCIDENT_TS': '12/31/2021 23:02:02',
 'OVERALL_EFF': '8386',
 'RESPOND_EFF': '4324',
 'DISPATCH_EFF': '317',
 'SOLVE_EFF': '3745'}

In [8]:
len(newsfeeds)

7401019

### 3.3  Create collection "Calls" in the apan5400 database to store NYPD calls 

In [27]:
collection = db.calls

In [38]:
%%time

# Insert Calls documents into collection Calls

collection.insert_many(newsfeeds)

CPU times: user 3min 8s, sys: 6min 40s, total: 9min 48s
Wall time: 28min 39s


In [28]:
collection.find_one()

{'_id': ObjectId('643efa7f31f75929ed8b8cae'),
 '': '1',
 'CAD_EVNT_ID': '82283653',
 'CREATE_DATE': '01/01/2022',
 'INCIDENT_DATE': '12/31/2021',
 'INCIDENT_TIME': '23:02:02',
 'NYPD_PCT_CD': '42',
 'BORO_NM': 'BRONX',
 'PATRL_BORO_NM': 'BRONX',
 'GEO_CD_X': '1014646',
 'GEO_CD_Y': '243583',
 'RADIO_CODE': '54E1',
 'TYP_DESC': 'AMBULANCE CASE: EDP/INSIDE',
 'CIP_JOBS': 'Non CIP',
 'ADD_TS': '01/01/2022 12:14:06 AM',
 'DISP_TS': '01/01/2022 12:19:23 AM',
 'ARRIVD_TS': 'NA',
 'CLOSNG_TS': '01/01/2022 01:21:48 AM',
 'Latitude': '40.83521',
 'Longitude': '-73.890157',
 'INCIDENT_TS': '12/31/2021 23:02:02',
 'OVERALL_EFF': '8386',
 'RESPOND_EFF': '4324',
 'DISPATCH_EFF': '317',
 'SOLVE_EFF': '3745'}

In [29]:
total_docs = collection.count_documents({})
total_docs

7401019

In [36]:
# collection.drop()

### 3.4 Execute MongoDB query

Same as above, the MongoDB query takes in three variables set as filters for $match. It returns the all the calls and their detailed information corresponding to the requirements.
For example, the following code returns all events happened on Jan. 1st, 2022, that was responded to by NYPD Precinct 42, and the incident includes fire.

In [30]:
%%time
nypd_var = '42'

pipeline = [ 
    { "$match": 
         { "NYPD_PCT_CD": nypd_var,
           "CREATE_DATE": '01/01/2022',
           "TYP_DESC":{"$regex":"FIRE",
                       "$options" :'i'}} 
    }
]
list(collection.aggregate(pipeline))[0]

CPU times: user 4.06 ms, sys: 4.84 ms, total: 8.89 ms
Wall time: 5.03 s


{'_id': ObjectId('643efa7f31f75929ed8b8d00'),
 '': '83',
 'CAD_EVNT_ID': '82284594',
 'CREATE_DATE': '01/01/2022',
 'INCIDENT_DATE': '01/01/2022',
 'INCIDENT_TIME': '00:06:56',
 'NYPD_PCT_CD': '42',
 'BORO_NM': 'BRONX',
 'PATRL_BORO_NM': 'BRONX',
 'GEO_CD_X': '1015519',
 'GEO_CD_Y': '241579',
 'RADIO_CODE': '10S2',
 'TYP_DESC': 'INVESTIGATE/POSSIBLE CRIME: SHOTS FIRED/OUTSIDE',
 'CIP_JOBS': 'Critical',
 'ADD_TS': '01/01/2022 12:06:56 AM',
 'DISP_TS': '01/01/2022 12:07:41 AM',
 'ARRIVD_TS': 'NA',
 'CLOSNG_TS': '01/01/2022 12:54:28 AM',
 'Latitude': '40.829707',
 'Longitude': '-73.887011',
 'INCIDENT_TS': '01/01/2022 00:06:56',
 'OVERALL_EFF': '2852',
 'RESPOND_EFF': '0',
 'DISPATCH_EFF': '45',
 'SOLVE_EFF': '2807'}

In [31]:
%%time
nypd_var = '42'
date_time = '01/01/2022'
type_incident = 'FIRE'

pipeline = [ 
    { "$match": 
         { "NYPD_PCT_CD": nypd_var,
           "CREATE_DATE": date_time,
           "TYP_DESC":{"$regex":type_incident,
                       "$options" :'i'}} 
    }
]
list(collection.aggregate(pipeline))[0]

CPU times: user 5.41 ms, sys: 2.85 ms, total: 8.26 ms
Wall time: 3.64 s


{'_id': ObjectId('643efa7f31f75929ed8b8d00'),
 '': '83',
 'CAD_EVNT_ID': '82284594',
 'CREATE_DATE': '01/01/2022',
 'INCIDENT_DATE': '01/01/2022',
 'INCIDENT_TIME': '00:06:56',
 'NYPD_PCT_CD': '42',
 'BORO_NM': 'BRONX',
 'PATRL_BORO_NM': 'BRONX',
 'GEO_CD_X': '1015519',
 'GEO_CD_Y': '241579',
 'RADIO_CODE': '10S2',
 'TYP_DESC': 'INVESTIGATE/POSSIBLE CRIME: SHOTS FIRED/OUTSIDE',
 'CIP_JOBS': 'Critical',
 'ADD_TS': '01/01/2022 12:06:56 AM',
 'DISP_TS': '01/01/2022 12:07:41 AM',
 'ARRIVD_TS': 'NA',
 'CLOSNG_TS': '01/01/2022 12:54:28 AM',
 'Latitude': '40.829707',
 'Longitude': '-73.887011',
 'INCIDENT_TS': '01/01/2022 00:06:56',
 'OVERALL_EFF': '2852',
 'RESPOND_EFF': '0',
 'DISPATCH_EFF': '45',
 'SOLVE_EFF': '2807'}

## 4. Develop User Interface using Python Flask

The user is expected to input the first two fields first to find the NYPDs. Then they could find the incidents they are interested in from the further three fields. 
The user cannot fill in all the five fields, either the first two or the last three. If they input all of them, the server will only return the results from RDBMS. 
The user does not have to fill in both the two fields, or the last three fields. They can simply search for the borough or simply NYPD precinct without other key inputes. The server will still return the corresponding results.
If the user does not input anything and clicks on submit, the server will ask the user to check his/her input.

In [53]:
from flask import Flask, request, render_template
app = Flask("APAN5400App")

@app.route('/')
def my_form():
    return render_template("my_form.html")

@app.route('/', methods=['POST'])
def my_form_post():
    var1 = request.form['userinput1']
    var2 = request.form['userinput2']
    var3 = request.form['userinput3']
    var4 = request.form['userinput4']
    var5 = request.form['userinput5']
    if var1 != '':
        queryCmd = "Select * from NYPD_PCT where BORO_NM LIKE '%%%s%%' and PATRL_BORO_NM LIKE '%%%s%%'"  % ( var1, var2 ) 
        cur.execute(queryCmd) 

        row = cur.fetchone()
        result = []
        while row is not None:
            result.append(row)
            row = cur.fetchone() 

        return str(result)
    elif var3 != '':
        nypd_var = var3
        date_time = var4
        type_incident = var5

        pipeline = [ 
            { "$match": 
                 { "NYPD_PCT_CD": nypd_var,
                   "CREATE_DATE": date_time,
                   "TYP_DESC":{"$regex":type_incident,
                               "$options" :'i'}} 
            }
        ]
        result = []
        for record in collection.aggregate(pipeline):
            result.append(record)

        return str(result)
    else:
        result = 'Please check your input.'
        return result


In [55]:
# Run flask
app.run(host='localhost', port=5011)

 * Serving Flask app 'APAN5400App'
 * Debug mode: off


 * Running on http://localhost:5011
Press CTRL+C to quit
127.0.0.1 - - [28/Apr/2023 11:44:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2023 11:44:00] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [28/Apr/2023 11:45:40] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2023 11:46:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2023 11:48:22] "POST / HTTP/1.1" 200 -


### 4.1 Input variable Try-out on Flask

In [196]:
@app.route('/')
def my_form():
    return render_template("my_form.html")

@app.route('/', methods=['POST'])
def my_form_post():
    nypd_var = request.form['userinput3']
    date_time = request.form['userinput4']
    type_incident = request.form['userinput5']

    pipeline = [ 
        { "$match": 
             { "NYPD_PCT_CD": nypd_var,
               "CREATE_DATE": date_time,
               "TYP_DESC":{"$regex":type_incident,
                           "$options" :'i'}} 
        }
    ]
    
    result = []
    for record in collection.aggregate(pipeline):
        result.append(record['CAD_EVNT_ID'])

    return str(result)

In [203]:
@app.route('/')
def my_form():
    return render_template("my_form.html")

@app.route('/', methods=['POST'])
def my_form_post():
    var1 = request.form['userinput1']
    var2 = request.form['userinput2']
    queryCmd_6 = "Select * from NYPD_PCT where BORO_NM LIKE '%%%s%%' and PATRL_BORO_NM LIKE '%%%s%%'"  % ( var1, var2 ) 
    cur.execute(queryCmd_6) 

    row = cur.fetchone()
    result = []
    while row is not None:
        result.append(row[2])
        row = cur.fetchone() 

    return str(result)



## The End!